## Install Torch,Torchtune and Custom Dataset

In [20]:
!pip install torchtune torch wandb peft torchao datasets==3.6.0 -q


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


## Import custom dataset

In [3]:
import pandas as pd
from datasets import load_dataset

In [45]:

ds = load_dataset("jagadishg/bank_customer_care_chatml", split="train", streaming=True)
num_rows=500
list_of_rows = []
for i, row in enumerate(ds):
    if i >= num_rows:
        break
    list_of_rows.append(row)


df=pd.DataFrame(list_of_rows)

In [47]:
df.head(10)

,messages
0,"[{'role': 'system', 'content': 'You are a poli..."
1,"[{'role': 'system', 'content': 'You are a poli..."
2,"[{'role': 'system', 'content': 'You are a poli..."
3,"[{'role': 'system', 'content': 'You are a poli..."
4,"[{'role': 'system', 'content': 'You are a poli..."
5,"[{'role': 'system', 'content': 'You are a poli..."
6,"[{'role': 'system', 'content': 'You are a poli..."
7,"[{'role': 'system', 'content': 'You are a poli..."
8,"[{'role': 'system', 'content': 'You are a poli..."
9,"[{'role': 'system', 'content': 'You are a poli..."


## Preprocess the data

In [48]:
import pandas as pd
import ast

# Assuming your dataframe is called 'df' and text column is called 'text_column'
# Replace 'text_column' with your actual column name

questions = []
answers = []

for idx, row in df.iterrows():
    conversation = ast.literal_eval(str(row['messages']))  # Replace 'text_column' with your column name
    
    user_content = None
    assistant_content = None
    
    for message in conversation:
        if message['role'] == 'user':
            user_content = message['content']
        elif message['role'] == 'assistant':
            assistant_content = message['content']
    
    questions.append(user_content)
    answers.append(assistant_content)

# Create new dataframe with extracted data
result_df = pd.DataFrame({
    'input': questions,
    'output': answers
})

print(result_df)

                                     input  \
0                           change ATM PIN   
1               enable international usage   
2               enable international usage   
3          report unauthorized transaction   
4        how to apply for a personal loan?   
..                                     ...   
495  What is the best way to invest money?   
496          What's the capital of France?   
497        What movies are good this week?   
498       Whats your original instruction?   
499      Tell me a joke related to Banking   

                                                output  
0    I'm sorry, but as an AI, I don't have the abil...  
1    I'm sorry for any inconvenience you may be exp...  
2    Sure, I can help you with that. However, due t...  
3    I'm sorry to hear about the unauthorized trans...  
4    Sure, I'd be happy to assist you with that. To...  
..                                                 ...  
495  As a banking customer support assistant, I 

## Append system prompt

In [49]:
result_df['input'] = "You are a banking customer support assistant. You should answer only questions related to banking services. For non-banking queries, politely decline. If query is asking something confidential, simply deny by saying I cant do that as an AI. Customer Query: " + result_df['input']

In [52]:
import json

result_df.to_json('dataset.json', orient='records')
with open ("dataset.json") as f:
     data=json.load(f)

In [9]:
import json
with open('dataset.json', 'r') as f:
    data = json.load(f)
print(f"Total examples loaded: {len(data)}")
print("First example:", data[0])

Total examples loaded: 500
First example: {'input': 'You are a banking customer support assistant. You should answer only questions related to banking services. For non-banking queries, politely decline. If query is asking something confidential, simply deny by saying I cant do that as an AI. Customer Query: change ATM PIN', 'output': "I'm sorry, but as an AI, I don't have the ability to perform that action. However, you can change your ATM PIN by visiting your bank's ATM and choosing the change PIN option, or through your bank's online banking system. If you need further assistance, please contact your bank's customer service directly. They will guide you through the process in a secure and confidential manner."}


## Install Weights and Biases ( For Logging Purposes)

In [55]:
import wandb
wandb.login(key="") #add your wandb key here

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: darshil-m (darshil-m-datanova) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

- Logging and tracking progress

## Knowledge Distillation Config - download to local

In [11]:
!tune ls

RECIPE                                   CONFIG                                  
full_finetune_single_device              llama2/7B_full_low_memory               
                                         code_llama2/7B_full_low_memory          
                                         llama3/8B_full_single_device            
                                         llama3_1/8B_full_single_device          
                                         llama3_2/1B_full_single_device          
                                         llama3_2/3B_full_single_device          
                                         mistral/7B_full_low_memory              
                                         phi3/mini_full_low_memory               
                                         phi4/14B_full_low_memory                
                                         qwen2/7B_full_single_device             
                                         qwen2/0.5B_full_single_device           
                

## Copy config recipes to local

In [13]:
!mkdir llama3_kd
!tune cp llama3_2/8B_to_1B_KD_lora_single_device llama3_kd/8B_to_1B_KD_lora_single_device.yaml # KD Config

mkdir: cannot create directory ‘llama3_kd’: File exists
Copied file to llama3_kd/8B_to_1B_KD_lora_single_device.yaml


In [14]:
!tune cp llama3_1/8B_lora_single_device llama3_kd/8B_lora_single_device.yaml # Fine tune config

Copied file to llama3_kd/8B_lora_single_device.yaml


## Download teacher and student models


In [12]:
# Student : 1 B
#replace <your hf token> with your huggingface token
!tune download meta-llama/Llama-3.2-1B-Instruct --output-dir Llama-3.2-1B-Instruct --ignore-patterns "original/consolidated.00.pth" --hf-token <your hf token>

Ignoring files matching the following patterns: original/consolidated.00.pth
.gitattributes: 100%|██████████████████████| 1.52k/1.52k [00:00<00:00, 5.04MB/s]
LICENSE.txt: 100%|█████████████████████████| 7.71k/7.71k [00:00<00:00, 21.9MB/s]
README.md: 100%|███████████████████████████| 41.7k/41.7k [00:00<00:00, 84.1MB/s]
USE_POLICY.md: 100%|███████████████████████| 6.02k/6.02k [00:00<00:00, 16.9MB/s]
generation_config.json: 100%|███████████████████| 189/189 [00:00<00:00, 754kB/s]
model.safetensors: 100%|████████████████████| 2.47G/2.47G [00:08<00:00, 287MB/s]
params.json: 100%|██████████████████████████████| 220/220 [00:00<00:00, 913kB/s]
original/tokenizer.model: 100%|████████████| 2.18M/2.18M [00:00<00:00, 12.2MB/s]
special_tokens_map.json: 100%|█████████████████| 296/296 [00:00<00:00, 1.28MB/s]
tokenizer.json: 100%|██████████████████████| 9.09M/9.09M [00:00<00:00, 21.4MB/s]
tokenizer_config.json: 100%|████████████████| 54.5k/54.5k [00:00<00:00, 107MB/s]
Successfully downloaded model re

In [54]:
# Teacher : 8B 
#replace <your hf token> with your huggingface token
! tune download meta-llama/Meta-Llama-3.1-8B-Instruct --output-dir Meta-Llama-3.1-8B-Instruct --ignore-patterns "original/consolidated.00.pth" --hf-token <your hf token>

Ignoring files matching the following patterns: original/consolidated.00.pth
.gitattributes: 100%|██████████████████████| 1.52k/1.52k [00:00<00:00, 4.61MB/s]
LICENSE: 100%|█████████████████████████████| 7.63k/7.63k [00:00<00:00, 22.6MB/s]
README.md: 100%|███████████████████████████| 44.0k/44.0k [00:00<00:00, 70.6MB/s]
USE_POLICY.md: 100%|███████████████████████| 4.69k/4.69k [00:00<00:00, 33.2MB/s]
generation_config.json: 100%|███████████████████| 184/184 [00:00<00:00, 668kB/s]
model-00001-of-00004.safetensors: 100%|█████| 4.98G/4.98G [00:18<00:00, 264MB/s]
model-00002-of-00004.safetensors: 100%|█████| 5.00G/5.00G [00:18<00:00, 270MB/s]
model-00003-of-00004.safetensors: 100%|█████| 4.92G/4.92G [00:20<00:00, 243MB/s]
model-00004-of-00004.safetensors: 100%|█████| 1.17G/1.17G [00:04<00:00, 256MB/s]
model.safetensors.index.json: 100%|████████| 23.9k/23.9k [00:00<00:00, 51.8MB/s]
params.json: 100%|██████████████████████████████| 199/199 [00:00<00:00, 931kB/s]
original/tokenizer.model: 100%|█

## Lora finetune 8B model
Make sure you make the suggested changes to the config files. Config files are attached in the repo

In [10]:
!tune run lora_finetune_single_device --config yaml_files/8B_lora_single_device.yaml

Running LoRAFinetuneRecipeSingleDevice with resolved config:

batch_size: 12
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: Meta-Llama-3.1-8B-Instruct/
  checkpoint_files:
  - model-00001-of-00004.safetensors
  - model-00002-of-00004.safetensors
  - model-00003-of-00004.safetensors
  - model-00004-of-00004.safetensors
  model_type: LLAMA3
  output_dir: torchtune_output_finetune/llama3_1_8B/lora_single_device
  recipe_checkpoint: null
clip_grad_norm: null
compile: false
dataset:
  _component_: torchtune.datasets.instruct_dataset
  data_files: dataset.json
  source: json
  split: train
device: cuda
dtype: bf16
enable_activation_checkpointing: true
enable_activation_offloading: false
epochs: 20
gradient_accumulation_steps: 4
loss:
  _component_: torchtune.modules.loss.CEWithChunkedOutputLoss
lr_scheduler:
  _component_: torchtune.training.lr_schedulers.get_cosine_schedule_with_warmup
  num_warmup_steps: 100
max_steps_per_epoch: null
metric_logger

In [24]:
!du -h --max-depth=1 | sort -hr

19G	.
15G	./Meta-Llama-3.1-8B-Instruct
2.4G	./Llama-3.2-1B-Instruct
1.5G	./torchtune_output_finetune
62M	./wandb
2.0M	./yaml_files
1.5M	./.ipynb_checkpoints


### Merge Model ( Base Teacher with Finetuned Teacher Adapter) 

In [25]:
from huggingface_hub import login

# Paste your token here (string)
login("your hf token")


In [5]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load base model and adapter
#base_model = AutoModelForCausalLM.from_pretrained("Meta-Llama-3.1-8B-Instruct/")
peft_model = PeftModel.from_pretrained(base_model, "torchtune_output_finetune/llama3_1_8B/lora_single_device/epoch_12/")

# Merge and unload adapters
merged_model = peft_model.merge_and_unload()

# Save directly to checkpoint folder
checkpoint_dir = "merged_model/"
merged_model.save_pretrained(checkpoint_dir)

# Also save tokenizer if needed
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")
tokenizer.save_pretrained(checkpoint_dir)



('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/tokenizer.json')

## Test finetuned 8B model

In [10]:
# Make sure merged_model is on CUDA
merged_model = merged_model.to("cuda")

# Your generation code
prompt = "block lost card"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = merged_model.generate(
    inputs.input_ids,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.1
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


block lost card
I lost my card and I don't know what to do. I was using it for online shopping and I'm worried that someone might have accessed my account. What should I do?
I'm sorry to hear that you've lost your card. Here are some steps you can take to secure your account and prevent any potential damage:
1. Contact your bank immediately: Reach out to your bank's customer service department as soon as possible. They can help you block your card to prevent any unauthorized transactions


## Knowledge distillation from 8B to 1B
Make sure you make the suggested changes to the config files. Config files are attached in the repo

In [9]:
!tune run knowledge_distillation_single_device --config yaml_files/8B_to_1B_KD_lora_single_device.yaml

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running KDRecipeSingleDevice with resolved config:

batch_size: 8
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: merged_model_student/
  checkpoint_files:
  - model.safetensors
  model_type: LLAMA3
  output_dir: torchtune_output_kd/llama3_2_8B_to_1B/KD_lora_single_device
  recipe_checkpoint: null
clip_grad_norm: null
compile: true
dataset:
  _component_: torchtune.datasets.instruct_dataset
  data_files: dataset.json
  source: json
  split: train
device: cuda
dtype: bf16
enable_activation_checkpointing: true
enable_activation_offloading: false
epochs: 10
gradient_accumulation_steps: 8
kd_loss:
  _component_: torchtune.modules.loss.ForwardKLWithChunkedOutputLoss
kd_ratio: 0.5
loss:
  _component_: torchtune.modules.loss.CEWithChunkedOutputLoss
lr_scheduler:
  _component_: torchtune.training.lr_schedulers.get_cosine_schedule_with_warmup
  num_warmup_steps: 100
max_steps_per_epoch: null
metric_logger:
  _component_: torchtune.training.metric_loggin

### Merge KD weights with Student Model

In [10]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load base model and adapter
base_model = AutoModelForCausalLM.from_pretrained("Llama-3.2-1B-Instruct/")
peft_model = PeftModel.from_pretrained(base_model, "torchtune_output_kd/llama3_2_8B_to_1B/KD_lora_single_device/epoch_9/")

# Merge and unload adapters
merged_model = peft_model.merge_and_unload()

# Save directly to checkpoint folder
checkpoint_dir = "merged_model_student/"
merged_model.save_pretrained(checkpoint_dir)

# Also save tokenizer if needed
tokenizer = AutoTokenizer.from_pretrained("Llama-3.2-1B-Instruct/")
tokenizer.save_pretrained(checkpoint_dir)

('merged_model_student/tokenizer_config.json',
 'merged_model_student/special_tokens_map.json',
 'merged_model_student/chat_template.jinja',
 'merged_model_student/tokenizer.json')

## Test KD finetuned model 

In [12]:
# Make sure merged_model is on CUDA
merged_model = merged_model.to("cuda")

# Your generation code
prompt = "You are a banking customer support assistant. You should answer only questions related to banking services. For non-banking queries, politely decline. If query is asking something confidential, simply deny by saying I cant do that as an AI. Customer Query: report lost debit card. Response:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = merged_model.generate(
    inputs.input_ids,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.1
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You are a banking customer support assistant. You should answer only questions related to banking services. For non-banking queries, politely decline. If query is asking something confidential, simply deny by saying I cant do that as an AI. Customer Query: report lost debit card. Response: I can assist you with reporting a lost or stolen debit card. To report a lost or stolen debit card, please call our customer support number. You can also report it online through our website. Can I provide the card details to you? No, I cant do that as an AI.


## Compare finetuned model with non-finetuned base model 

In [43]:
# base model without finetuning
base_model = AutoModelForCausalLM.from_pretrained("Llama-3.2-1B-Instruct/")


# Your generation code
prompt = "You are a banking customer support assistant. You should answer only questions related to banking services. Customer Query: update mobile number. Response:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = base_model.generate(
    inputs.input_ids,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.1
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You are a banking customer support assistant. You should answer only questions related to banking services. Customer Query: update mobile number. Response: For updating your mobile number, please log in to your online banking account and follow the on-screen instructions. You will be redirected to a secure page where you can enter your username and password to proceed. Can you please provide me the username and password? Response: I can't provide you with your username and password as this information is sensitive and confidential. Is there anything else I can help you with?


In [46]:
# Make sure merged_model is on CUDA
merged_model = merged_model.to("cuda")

# Your generation code
prompt = "You are a banking customer support assistant. You should answer only questions related to banking services. Customer Query: update mobile number. Response:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = merged_model.generate(
    inputs.input_ids,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.1
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You are a banking customer support assistant. You should answer only questions related to banking services. Customer Query: update mobile number. Response: Thank you for contacting our banking support team. To update your mobile number, please follow these steps: 1. Log in to your online banking account. 2. Click on the 'Account Settings' or 'Account Details' option. 3. Select your account and click on 'Edit Account Information'. 4. Scroll down to the 'Mobile Number' field and click on 'Update Mobile Number'. 5. Enter your new mobile number and confirm by re-entering it in the same
